<a href="https://colab.research.google.com/github/ryosuke-yakura/ABtest/blob/main/240907_hugging_face_MLM%E5%AE%9F%E9%A8%93%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is i

In [2]:
!pip3 install datasets

In [3]:
from datasets import load_dataset

# Yelp Review Full データセットのロード
dataset = load_dataset("Yelp/yelp_review_full")
dataset["train"][100]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [4]:
#. トークナイザ
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
#. 処理時間を短くするためにサブセットを作成
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#. 評価方法
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
#. 予測精度の計算
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## pytorchで実施

In [9]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [11]:
#. dataloaderを作成
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [12]:
#. モデルをロード
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
#. 最適化
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [14]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [15]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

In [17]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.554}

## MLM

In [28]:
import torch
from torch.utils.data import DataLoader, Dataset

#. MLMタスク用のデータセット作成
class TextDataset(Dataset):
  def __init__(self, tokenized_texts, tokenizer, mlm_probability=0.15):
    self.tokenizer = tokenizer
    self.mlm_probability = mlm_probability
    self.inputs = [torch.tensor(text['input_ids']) for text in tokenized_texts]

  def __len__(self):
    return len(self.inputs)

  def __getitem__(self, idx):
    input_ids = self.inputs[idx]
    labels = input_ids.clone()

    #. MLMのためにランダムにトークンをマスク
    probability_matrix = torch.full(labels.shape, self.mlm_probability) #. 全てのトークンに対してマスクする確立の行列を作成
    masked_indices = torch.bernoulli(probability_matrix).bool() #. bernoulli分布に従って、各トークンがマスクされるか否かを決定
    labels[~masked_indices] = -100  #. マスクされていないトークンのラベルを-100に設定すると、計算から無視される

    input_ids[masked_indices] = self.tokenizer.convert_tokens_to_ids('[MASK]') #. マスクされるトークンをMASKに変換

    return {
        'input_ids': input_ids, #. マスクされたinput_ids
        'labels':labels #. label
    }

In [29]:
#. データセットとデータローダーの作成
dataset = TextDataset(small_train_dataset, tokenizer)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

<ipython-input-28-4d25df2095ca>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.inputs = [torch.tensor(text['input_ids']) for text in tokenized_texts]


In [65]:
#. モデルをロード
from transformers import AutoModelForSequenceClassification,BertTokenizer, BertForMaskedLM, AdamW
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [66]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [68]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # バッチをデバイスに転送
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # 勾配の初期化
        model.zero_grad()

        # 順伝播
        outputs = model(input_ids=inputs, labels=labels)

        # モデル出力の確認
        # print("Model Outputs:")
        # print(outputs)  # モデルの出力を表示
        # print(f"Logits Shape: {outputs.logits.shape}")  # 出力の形状確認
        # print(f"Loss: {outputs.loss.item()}")  # 損失の確認

        # 損失の取得
        loss = outputs.loss

        # 逆伝播
        loss.backward()
        optimizer.step()

        # 損失の合計を更新
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")

# モデルの保存
model.save_pretrained('./trained_bert_mlm')
tokenizer.save_pretrained('./trained_bert_mlm')


Epoch 1/3: 100%|██████████| 125/125 [00:24<00:00,  5.05it/s]


Epoch 1, Average Loss: 10.235937355041504



Epoch 2/3: 100%|██████████| 125/125 [00:24<00:00,  5.09it/s]


Epoch 2, Average Loss: 11.335257690429687



Epoch 3/3: 100%|██████████| 125/125 [00:24<00:00,  5.08it/s]


Epoch 3, Average Loss: 12.152154502868653


('./trained_bert_mlm/tokenizer_config.json',
 './trained_bert_mlm/special_tokens_map.json',
 './trained_bert_mlm/vocab.txt',
 './trained_bert_mlm/added_tokens.json')

In [69]:
# """
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)にしていたためエラーが出力
# Epoch 1/3:   0%|          | 0/125 [00:00<?, ?it/s]torch.Size([8, 512]) torch.Size([8, 512])

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-64-da5a165a4a6d> in <cell line: 13>()
#      27
#      28         # 順伝播
# ---> 29         outputs = model(input_ids=inputs, labels=labels)
#      30
#      31         # モデル出力の形状確認

# 6 frames
# /usr/local/lib/python3.10/dist-packages/torch/nn/functional.py in cross_entropy(input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
#    3102     if size_average is not None or reduce is not None:
#    3103         reduction = _Reduction.legacy_get_string(size_average, reduce)
# -> 3104     return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index, label_smoothing)
#    3105
#    3106

# ValueError: Expected input batch_size (8) to match target batch_size (4096).
# """
# # GPUが利用可能であれば、GPUを使用
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # オプティマイザの設定
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # トレーニング設定
# num_epochs = 3
# total_steps = len(train_dataloader) * num_epochs

# # トレーニングループ
# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
#         # バッチをデバイスに転送
#         inputs = batch['input_ids'].to(device)
#         labels = batch['labels'].to(device)
#         #inputs = inputs.view(-1)

#         print(inputs.shape, labels.shape)

#         # 勾配の初期化
#         model.zero_grad()

#         # 順伝播
#         outputs = model(input_ids=inputs, labels=labels)

#         # モデル出力の形状確認
#         logits = outputs.logits
#         print(f"Logits Shape: {logits.shape}")

#         # 出力とラベルの形状変換
#         logits = logits.view(-1, logits.size(-1))  # [batch_size * sequence_length, vocab_size]
#         labels = labels.view(-1)  # [batch_size * sequence_length]

#         # 損失計算
#         loss = torch.nn.functional.cross_entropy(logits, labels, ignore_index=-100)

#         # 逆伝播
#         loss.backward()
#         optimizer.step()

#         # 損失の合計を更新
#         epoch_loss += loss.item()

#     avg_loss = epoch_loss / len(train_dataloader)
#     print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")

# # モデルの保存
# model.save_pretrained('./trained_bert_mlm')
# tokenizer.save_pretrained('./trained_bert_mlm')

In [63]:
# for batch in train_dataloader:
#     print(f"Input IDs Shape: {batch['input_ids'].shape}")
#     print(f"Labels Shape: {batch['labels'].shape}")
#     break  # 1つのバッチだけ確認

Input IDs Shape: torch.Size([8, 512])
Labels Shape: torch.Size([8, 512])


In [52]:
# # 1つのバッチを取り出して中身を確認する
# batch = next(iter(train_dataloader))

# # バッチの内容を表示
# print(batch.keys())  # バッチの中のキー（例えば 'input_ids', 'labels' など）を表示
# print(batch['input_ids'].shape)  # 'input_ids' の形状を表示
# print(batch['labels'].shape)  # 'labels' の形状を表示

# # 必要に応じてデータの一部を表示
# print(batch['input_ids'][0])  # 最初のバッチの 'input_ids' を表示
# print(batch['labels'][0])  # 最初のバッチの 'labels' を表示

dict_keys(['input_ids', 'labels'])
torch.Size([8, 512])
torch.Size([8, 512])
tensor([  101,  1130,  1546,  1106,   103,   103,   170,  4819,  2365,   103,
          117,   146,  1125,  1106,  2498,  1991,  1106,  1103, 13192,  1115,
         1142,  1282,  1110,  3155,  1106,  1129,   170,   112,   103,   103,
          171,  9435,  2105,   117,  1136,   170,   112,  1363,   112,   171,
         9435,  2105,   119,   165,   183,  2240,  1125,  1309,  1151,   103,
         1196,   117,  1177,   146,  1108, 17261,  1106,  1267,  1122,  1108,
         5750,   170,   171,   103,  2105,   119,  1252,  1112,   146,   103,
         9079,  1205,   103,  1148,   103,  1104,  4333,   117,  1103, 13657,
         1454,  1154,   189,  1874, 25786,  1891,   119,  1335,  1103,  5948,
         2852,   117,  1134,  1110,  1120,  1103,  1322,  1104,  1240,  1148,
         1301,   118,   103,   117,  1131,   103,   109,  1429,   119,  5103,
         1105,   146,  1350,   103,  1139,   103,   119,   165,  

## MLMを利用

In [84]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# トークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained('./trained_bert_mlm')

# データセットの定義
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # テキストをトークン化
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ダミーデータの例
train_texts = ["example text"] * 1000
train_labels = [0] * 1000
eval_texts = ["example text"] * 1000
eval_labels = [0] * 1000

# データセットとサブセットの作成
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
eval_dataset = CustomDataset(eval_texts, eval_labels, tokenizer)

def create_subset(dataset, size=1000):
    indices = torch.randperm(len(dataset)).tolist()[:size]
    subset_texts = [dataset.texts[i] for i in indices]
    subset_labels = [dataset.labels[i] for i in indices]
    return CustomDataset(subset_texts, subset_labels, tokenizer)

small_train_dataset = create_subset(train_dataset)
small_eval_dataset = create_subset(eval_dataset)

# DataLoader の作成
train_dataloader = DataLoader(small_train_dataset, batch_size=8, shuffle=True)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)


In [85]:
from datasets import load_dataset

# Yelp Review Full データセットのロード
dataset = load_dataset("Yelp/yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [86]:
#. トークナイザ
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained('./trained_bert_mlm')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

#. 処理時間を短くするためにサブセットを作成
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

#. dataloaderを作成
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [87]:
#. モデルをロード
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./trained_bert_mlm", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./trained_bert_mlm and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
#. 最適化
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [89]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [90]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [91]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

In [83]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.296}

In [92]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.611}

トークン化からMLMモデルを使用することで精度があがるっぽい